In [51]:
from model import DeepPose
from train import *
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
from train import GaussianNoise
from cuboid_pnp_solver import CuboidPNPSolver
from object_detector import ObjectDetector
import json
from cuboid import Cuboid3d

In [38]:
checkpoint_path = 'checkpoint.pth'
model = DeepPose()
load_model(checkpoint_path, model)

=> Loading checkpoint
=> Loaded checkpoint


In [41]:
from dataloader import *

root = 'test_data'
modelsPath = 'lm_models/models/models_info.json'
annFile = 'LOOKHEREannotations.json'
transform = transforms.Compose([
    transforms.ToTensor(),
    GaussianNoise(mean=0.0, std=.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
])
normalize = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225), inplace=True),
    ])
dataset = LineMODCocoDataset(root, annFile, modelsPath, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [45]:
img, target = dataset[0]

In [49]:
cam_json_file = 'lm/camera.json'
with open(cam_json_file, 'r') as f:
    camera_params = json.load(f)

cx = camera_params['cx']
cy = camera_params['cy']
fx = camera_params['fx']
fy = camera_params['fy']

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
])

In [ ]:
solver = CuboidPNPSolver(camera_intrinsic_matrix=K, )
img_norm = normalize(dataset[0][0])

In [ ]:
def run_inference(model, dataloader):
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for img, target in dataloader:
            img = img.to(device)
            output = model(img)
            

            # Assuming output contains the predicted pose
            predicted_pose = pnp(output)
            ground_truth_pose = target['pose']
            
            all_predictions.append(predicted_pose.cpu().numpy())
            all_targets.append(ground_truth_pose.cpu().numpy())
            
    return all_predictions, all_targets


In [ ]:
import numpy as np

def compute_add(vertices, pred_pose, gt_pose):
    # Transform vertices with ground truth pose
    vertices_gt = (gt_pose[:3, :3] @ vertices.T + gt_pose[:3, 3:4]).T
    
    # Transform vertices with predicted pose
    vertices_pred = (pred_pose[:3, :3] @ vertices.T + pred_pose[:3, 3:4]).T
    
    # Compute Euclidean distances
    distances = np.linalg.norm(vertices_gt - vertices_pred, axis=1)
    
    # Return average distance
    return np.mean(distances)


In [ ]:
# Assuming `model` is your trained model and `vertices` contains 3D points of the object model
device = torch.device('mps')
model.to(device)

predictions, targets = run_inference(model, dataloader)

add_scores = []
for pred_pose, gt_pose, target in zip(predictions, targets, dataset):
    vertices = dataset.create_3D_vertices(target['category_id'])
    add = compute_add(vertices, pred_pose, gt_pose)
    add_scores.append(add)

# Compute average ADD score
average_add = np.mean(add_scores)
print(f'Average ADD: {average_add}')


In [ ]:
img = dataset[0][0].numpy()
img

In [ ]:
# plt.imshow(dataset[0][0])
plt.imshow(img.transpose(1, 2, 0))

In [ ]:
import numpy as np

images, targets = next(iter(dataloader))
first_image = images[0]
first_target = targets[0]



In [ ]:


numpy_first_image = first_image.numpy()
plt.imshow(numpy_first_image)

In [ ]:
plt.imshow(first_target[0])

In [ ]:
# evaluate model on the first image/target pair

model.eval()
with torch.no_grad():
    pred_belief_maps, pred_vector_fields = model(first_image.unsqueeze(0))

In [ ]:
batch_size = pred_belief_maps.shape[0]
num_belief_maps = pred_belief_maps.shape[2]

# Plot each belief map in the batch
for i in range(batch_size):
    for j in range(num_belief_maps):
        belief_map = pred_belief_maps[i, 0, j].cpu().numpy()

        plt.figure(figsize=(6, 5))
        plt.imshow(belief_map, cmap='hot')
        plt.colorbar()
        plt.title(f'Belief Map {j+1} (Batch {i+1})')
        plt.show()

In [ ]:
pred_vector_fields.shape